In [1]:
import tensorflow as tf

In [3]:
%pwd

'c:\\Users\\rahul\\Downloads\\sign_language_conversion\\research'

In [4]:
import os 
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\rahul\\Downloads\\sign_language_conversion'

In [7]:
model = tf.keras.models.load_model("model/model.h5")

In [15]:
import os

def get_classes_from_dataset(dataset_path):
    """
    Traverses the dataset directory and creates an array of class names.
    
    Args:
    - dataset_path (str): Path to the dataset directory.
    
    Returns:
    - class_names (list): List of class names.
    """
    # Check if the dataset path exists
    if not os.path.exists(dataset_path):
        raise ValueError(f"Dataset path '{dataset_path}' does not exist.")
    
    # List all subdirectories in the dataset path
    class_names = []
    for item in os.listdir(dataset_path):
        item_path = os.path.join(dataset_path, item)
        if os.path.isdir(item_path):
            class_names.append(item)
    
    return class_names

# Example usage
dataset_path = 'artifacts\data_ingestion\dataset\Train'
class_names = get_classes_from_dataset(dataset_path)
print("Classes in the dataset:", class_names)

Classes in the dataset: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'Nothing', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [16]:
import cv2
     

test_image = cv2.imread('artifacts\data_ingestion\dataset\Test\Y\3004.jpg')
     

plt.imshow(test_image)
     

test_image.shape
     

test_image = cv2.resize(test_image,(224,224))
     

test_input = test_image.reshape(1,224,224,3)
     

model.predict(test_input)

ModuleNotFoundError: No module named 'cv2'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import os

batch_size = 32 

train_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.1, 0.4),
    zoom_range=0.4,
    channel_shift_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True,
    vertical_flip=True,
)

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_generator.flow_from_directory(
    '/teamspace/studios/this_studio/dataset/Train',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical"
)

test_data = test_generator.flow_from_directory(
    '/teamspace/studios/this_studio/dataset/Test',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

pretrained_model = VGG16(include_top=False, input_shape=input_shape, weights='imagenet', input_tensor=inputs)

for layer in pretrained_model.layers:
    layer.trainable = False

block_output = pretrained_model.get_layer('block5_pool').output

x = Flatten()(block_output)

x = Dense(512, activation='relu', kernel_initializer='he_uniform')(x)
x = Dropout(0.4)(x)
x = Dense(216, activation='relu', kernel_initializer='he_uniform')(x)
output = Dense(28, activation='softmax', kernel_initializer='he_uniform')(x)

model = Model(inputs=inputs, outputs=output)

model.load_weights('checkpoints/weights.008-0.9911.weights.h5')

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])